## **Codenation 2020**

**Nome**: Camila Morais de Melo <br>
**E-mail**: camila_moraismelo@hotmail.com 

**Desafio:** Descubra as melhores notas de matemática do ENEM 2016

**Pontos de Atenção:** Muitas universidades brasileiras utilizam o ENEM para selecionar seus futuros alunos e alunas. Isto é feito com uma média ponderada das notas das provas de matemática, ciências da natureza, linguagens e códigos, ciências humanas e redação, com os pesos abaixo:

* matemática: 3
*ciências da natureza: 2
*linguagens e códigos: 1.5
*ciências humanas: 1
*redação: 3

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna **NU_NOTA_MT**) de quem participou do ENEM 2016.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: **NU_INSCRICAO** e **NU_NOTA_MT**.

Faça o upload do arquivo answer.csv usando o botão “Submeter resposta”.

## **Setup**

In [29]:
# Pandas nos permite trabajar con DataFrames
import pandas as pd
# Para la visualización de datos
import plotly.express as px
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# model
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier ,VotingClassifier

# roc curve and auc score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

# show Colab Tables
from IPython.display import display
pd.set_option('display.max_columns', 100)
pd.options.display.max_columns = 100

#%matplotlib inline
#%load_ext google.colab.data_table

In [0]:
#!wget https://s3-us-west-1.amazonaws.com/codenation-challenges/enem-ps/testfiles.zip
#!unzip -o testfiles.zip

--2020-04-02 03:29:23--  https://s3-us-west-1.amazonaws.com/codenation-challenges/enem-ps/testfiles.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.136
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2694015 (2.6M) [application/zip]
Saving to: ‘testfiles.zip.1’

testfiles.zip.1     100%[===================>]   2.57M  5.27MB/s    in 0.5s    

2020-04-02 03:29:24 (5.27 MB/s) - ‘testfiles.zip.1’ saved [2694015/2694015]

Archive:  testfiles.zip
  inflating: test.csv                
  inflating: README.md               
  inflating: __MACOSX/._README.md    
  inflating: train.csv               


In [0]:
!ls

__MACOSX   sample_data	testfiles.zip	 train.csv
README.md  test.csv	testfiles.zip.1


## **Exploratório**

In [5]:
## Funções
def ler_arquivos(arquivo):
  df = pd.read_csv(arquivo)  
  print("Dimenções ",arquivo, "(filas, columnas)  : ", df.shape)
  return df

def remover_valores(df,column):
  df = df[(df[column].notnull())]
  print(df.shape)
  return df

def missing_values(df_train,df_test):
  percent_missing_train = df_train.isnull().sum() * 100 / len(df_train)
  percent_missing_test = df_test.isnull().sum() * 100 / len(df_test)
  missing_value_df = pd.DataFrame({'column_name': df_train.columns,
                                  'percent_missing_train': percent_missing_train,
                                  'percent_missing_test': percent_missing_test}).round(2)

  missing_value_df.sort_values('percent_missing_train', inplace=True)
  return missing_value_df

In [6]:
#Importa os dataframes e apresenta as dimenções 
df_training = ler_arquivos('train.csv')
df_test = ler_arquivos('test.csv')

('Dimen\xc3\xa7\xc3\xb5es ', 'train.csv', '(filas, columnas)  : ', (13730, 167))
('Dimen\xc3\xa7\xc3\xb5es ', 'test.csv', '(filas, columnas)  : ', (4576, 47))


In [7]:
#Manter apenas as colunas existentes no modelo de teste
#Neste caso fiz isso pois as dimenções eram diferentes
col = df_test.columns
#Acrescenta a variavel target
col = col.append(pd.Index(["NU_NOTA_MT"]))

df_training = df_training.filter(col)
print(df_training.shape)
print(df_test.shape)

(13730, 48)
(4576, 47)


In [8]:
cat_var = [key for key in dict(df_training.dtypes)
             if dict(df_training.dtypes)[key] in ['object'] ]

cat_var

['Q025',
 'Q024',
 'Q026',
 'Q047',
 'CO_PROVA_MT',
 'Q027',
 'CO_PROVA_CN',
 'Q006',
 'Q002',
 'SG_UF_RESIDENCIA',
 'Q001',
 'CO_PROVA_LC',
 'NU_INSCRICAO',
 'CO_PROVA_CH',
 'TP_SEXO']

In [9]:
ms = missing_values(df_training,df_test)
ms

C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\series.py:1999: RuntimeWarning:

invalid value encountered in rint



,column_name,percent_missing_test,percent_missing_train
CO_PROVA_CH,NU_INSCRICAO,0.00,0.00
Q006,NU_NOTA_CN,0.00,0.00
Q024,NU_NOTA_CH,0.00,0.00
Q025,NU_NOTA_LC,0.00,0.00
Q026,TP_LINGUA,0.00,0.00
Q047,NU_NOTA_COMP1,0.00,0.00
SG_UF_RESIDENCIA,NU_NOTA_COMP2,0.00,0.00
TP_ANO_CONCLUIU,NU_NOTA_COMP3,0.00,0.00
TP_COR_RACA,NU_NOTA_COMP4,0.00,0.00
TP_ESCOLA,Q001,0.00,0.00


In [11]:
#Transforma features categoricas em numericas
lb_make = LabelEncoder()
df_training["Q001"] = lb_make.fit_transform(df_training["Q001"])
df_test["Q001"] = lb_make.transform(df_test["Q001"])

lb_make = LabelEncoder()
df_training["Q002"] = lb_make.fit_transform(df_training["Q002"])
df_test["Q002"] = lb_make.transform(df_test["Q002"])

lb_make = LabelEncoder()
df_training["Q006"] = lb_make.fit_transform(df_training["Q006"])
df_test["Q006"] = lb_make.transform(df_test["Q006"])

lb_make = LabelEncoder()
df_training["Q024"] = lb_make.fit_transform(df_training["Q024"])
df_test["Q024"] = lb_make.transform(df_test["Q024"])

lb_make = LabelEncoder()
df_training["Q025"] = lb_make.fit_transform(df_training["Q025"])
df_test["Q025"] = lb_make.transform(df_test["Q025"])

lb_make = LabelEncoder()
df_training["Q026"] = lb_make.fit_transform(df_training["Q026"])
df_test["Q026"] = lb_make.transform(df_test["Q026"])

lb_make = LabelEncoder()
df_training["Q047"] = lb_make.fit_transform(df_training["Q047"])
df_test["Q047"] = lb_make.transform(df_test["Q047"])

lb_make = LabelEncoder()
df_training["TP_SEXO"] = lb_make.fit_transform(df_training["TP_SEXO"])
df_test["TP_SEXO"] = lb_make.transform(df_test["TP_SEXO"])

lb_make = LabelEncoder()
df_training["SG_UF_RESIDENCIA"] = lb_make.fit_transform(df_training["SG_UF_RESIDENCIA"])
df_test["SG_UF_RESIDENCIA"] = lb_make.transform(df_test["SG_UF_RESIDENCIA"])

lb_make = LabelEncoder()
df_training["CO_PROVA_CN"] = lb_make.fit_transform(df_training["CO_PROVA_CN"])
df_test["CO_PROVA_CN"] = lb_make.transform(df_test["CO_PROVA_CN"])

lb_make = LabelEncoder()
df_training["CO_PROVA_CH"] = lb_make.fit_transform(df_training["CO_PROVA_CH"])
df_test["CO_PROVA_CH"] = lb_make.transform(df_test["CO_PROVA_CH"])

lb_make = LabelEncoder()
df_training["CO_PROVA_LC"] = lb_make.fit_transform(df_training["CO_PROVA_LC"])
df_test["CO_PROVA_LC"] = lb_make.transform(df_test["CO_PROVA_LC"])

lb_make = LabelEncoder()
df_training["CO_PROVA_MT"] = lb_make.fit_transform(df_training["CO_PROVA_MT"])
df_test["CO_PROVA_MT"] = lb_make.transform(df_test["CO_PROVA_MT"])

In [12]:
#Cria valor NA para P:Com que idade você começou a exercer uma atividade remunerada?
df_training['Q027'] = df_training.Q027.fillna("NA")
df_test['Q027'] = df_test.Q027.fillna("NA")

lb_make = LabelEncoder()
df_training["Q027"] = lb_make.fit_transform(df_training["Q027"])
df_test["Q027"] = lb_make.transform(df_test["Q027"])

In [13]:
#Substitui o NA por um valor negativo
df_training['Q027'] = df_training.Q027.replace(to_replace=13,value=-1)
df_test['Q027'] = df_test.Q027.replace(to_replace=13,value=-1)

df_training.Q027.unique()

array([ 7, -1,  5,  4, 12,  2,  3,  1,  9,  0,  8,  6, 10, 11],
      dtype=int64)

In [14]:
cat_var = [key for key in dict(df_training.dtypes)
             if dict(df_training.dtypes)[key] in ['object'] ]

print(cat_var)

['NU_INSCRICAO']


## **Tratamento dos nulos e enriquecimento dos dados**

In [15]:
df_training.TP_ENSINO.value_counts(normalize=True)

1.0    0.907520
3.0    0.086408
2.0    0.006072
Name: TP_ENSINO, dtype: float64

In [16]:
df_training['TP_ENSINO'] = df_training.TP_ENSINO.fillna(1)
df_test['TP_ENSINO'] = df_test.TP_ENSINO.fillna(1)

df_training.TP_ENSINO.value_counts(normalize=True)

1.0    0.971158
3.0    0.026948
2.0    0.001894
Name: TP_ENSINO, dtype: float64

In [17]:
#Notas NA são informadas como -1 pois não foram informadas e há notas atualmente como 0 para features aqui atualizadas
df_training['NU_NOTA_CH'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_CH'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_CN'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_CN'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_LC'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_LC'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_REDACAO'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_REDACAO'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_COMP5'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_COMP5'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_COMP4'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_COMP4'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_COMP3'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_COMP3'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_COMP2'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_COMP2'] = df_training.TP_ENSINO.fillna(-1)

df_training['NU_NOTA_COMP1'] = df_training.TP_ENSINO.fillna(-1)
df_test['NU_NOTA_COMP1'] = df_training.TP_ENSINO.fillna(-1)

df_training['TP_STATUS_REDACAO'] = df_training.TP_ENSINO.fillna(-1)
df_test['TP_STATUS_REDACAO'] = df_training.TP_ENSINO.fillna(-1)

#Nota mínima determinada para não gerar no modelo nota negativa
df_training['NU_NOTA_MT'] = df_training.NU_NOTA_MT.fillna(0)

In [18]:
# TP_DEPENDENCIA_ADM_ESC: Dependência administrativa (Escola)
# 1	Federal
# 2	Estadual
# 3	Municipal
# 4	Privada
# NOVO: 5 NA
#
# TP_ESCOLA: Tipo de escola do Ensino Médio
# 1	Não Respondeu
# 4	Exterior

#Considerar que os valores faltantes são de Escolas Estaduais
df_training['TP_DEPENDENCIA_ADM_ESC'] = df_training.TP_DEPENDENCIA_ADM_ESC.fillna(2)
df_test['TP_DEPENDENCIA_ADM_ESC'] = df_training.TP_DEPENDENCIA_ADM_ESC.fillna(2)

In [19]:
ms = missing_values(df_training,df_test)
ms

,column_name,percent_missing_test,percent_missing_train
CO_PROVA_CH,NU_INSCRICAO,0.0,0.0
Q001,CO_PROVA_LC,0.0,0.0
Q002,CO_PROVA_MT,0.0,0.0
Q006,NU_NOTA_CN,0.0,0.0
Q024,NU_NOTA_CH,0.0,0.0
Q025,NU_NOTA_LC,0.0,0.0
Q026,TP_LINGUA,0.0,0.0
Q027,TP_STATUS_REDACAO,0.0,0.0
Q047,NU_NOTA_COMP1,0.0,0.0
SG_UF_RESIDENCIA,NU_NOTA_COMP2,0.0,0.0


In [0]:
#lb_make = LabelEncoder()
#df_training["NU_INSCRICAO"] = lb_make.fit_transform(df_training["NU_INSCRICAO"])
#df_test["NU_INSCRICAO"] = lb_make.fit_transform(df_test["NU_INSCRICAO"])

## **Modelo**

### **Linear Regression**

In [20]:
X = df_training
y = df_training['NU_NOTA_MT']

X = X.drop(['NU_NOTA_MT'],axis=1)
X = X.drop(['NU_INSCRICAO'],axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [22]:
y_pred = linreg.predict(X_test)

In [30]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

76.44923463293144


#### **Submission**

In [31]:
df_test.shape

(4576, 47)

In [32]:
df = df_test.drop(['NU_INSCRICAO',],axis=1)
pred = linreg.predict(df)

In [33]:
sub= pd.Series(pred, index=df_test['NU_INSCRICAO'].astype(np.str), name='NU_NOTA_MT')
sub.shape

(4576L,)

In [34]:
sub.head()

NU_INSCRICAO
73ff9fcc02f0a99919906c942c2e1a1042cdcf98    459.291181
71a95f9f1b91a82c65ad94abbdf9f54e6066f968    461.643769
b38a03232f43b11c9d0788abaf060f7366053b6d    505.264719
70b682d9a3636be23f6120fa9d6b164eb3c6002d     31.894690
715494628a50142ce8cb17191cfe6d0f3cae0934    527.408620
Name: NU_NOTA_MT, dtype: float64

### **Lightgbm**

In [35]:
def get_data_splits(dataframe, valid_fraction=0.1):
    valid_size = int(len(dataframe) * valid_fraction)

    train = dataframe[:-valid_size * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_size * 2:-valid_size]
    test = dataframe[-valid_size:]
    
    return train, valid, test

train, valid, test = get_data_splits(df_training)

In [37]:
import lightgbm as lgb

feature_cols = train.columns.drop(['NU_INSCRICAO','NU_NOTA_MT'])

dtrain = lgb.Dataset(train[feature_cols], label=train['NU_NOTA_MT'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['NU_NOTA_MT'])
dtest  = lgb.Dataset(test[feature_cols],  label=test['NU_NOTA_MT'])

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'verbose': 0
}

bst = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=dvalid, early_stopping_rounds=100)

[1]	valid_0's l2: 42220.3	valid_0's l1: 168.231
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 35288.7	valid_0's l1: 152.91
[3]	valid_0's l2: 29680.7	valid_0's l1: 139.627
[4]	valid_0's l2: 25134.2	valid_0's l1: 128.12
[5]	valid_0's l2: 21416.3	valid_0's l1: 118.137
[6]	valid_0's l2: 18429.9	valid_0's l1: 109.683
[7]	valid_0's l2: 16008.8	valid_0's l1: 102.34
[8]	valid_0's l2: 14053.9	valid_0's l1: 95.9926
[9]	valid_0's l2: 12445.2	valid_0's l1: 90.4676
[10]	valid_0's l2: 11155.5	valid_0's l1: 85.7634
[11]	valid_0's l2: 10083.4	valid_0's l1: 81.5381
[12]	valid_0's l2: 9219.34	valid_0's l1: 77.8667
[13]	valid_0's l2: 8528.18	valid_0's l1: 74.6599
[14]	valid_0's l2: 7961.77	valid_0's l1: 71.8589
[15]	valid_0's l2: 7494	valid_0's l1: 69.3737
[16]	valid_0's l2: 7109.98	valid_0's l1: 67.1991
[17]	valid_0's l2: 6792.28	valid_0's l1: 65.2248
[18]	valid_0's l2: 6544.98	valid_0's l1: 63.5416
[19]	valid_0's l2: 6340.31	valid_0's l1: 62.0271
[20]	valid_0's l2: 617

In [0]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

ypred = bst.predict(test[feature_cols], num_iteration=bst.best_iteration)
print('The rmse of prediction is:', mean_squared_error(test['NU_NOTA_MT'], ypred) ** 0.5)
#The rmse of prediction is: 70.38302047581291

The rmse of prediction is: 70.38302047581291


In [38]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

ypred = bst.predict(test[feature_cols], num_iteration=bst.best_iteration)
print('The rmse of prediction is:', mean_squared_error(test['NU_NOTA_MT'], ypred) ** 0.5)
#The rmse of prediction is: 70.38302047581291

('The rmse of prediction is:', 70.38302047581291)


#### **Submission**


In [39]:
feature_cols = df_test.columns.drop('NU_INSCRICAO')
len(feature_cols)

46

In [40]:
ypred = bst.predict(df_test[feature_cols], num_iteration=bst.best_iteration)

In [41]:
sub= pd.Series(ypred, index=df_test['NU_INSCRICAO'].astype(np.str), name='NU_NOTA_MT')
sub.shape

(4576L,)

In [42]:
sub.head()

NU_INSCRICAO
73ff9fcc02f0a99919906c942c2e1a1042cdcf98    473.178365
71a95f9f1b91a82c65ad94abbdf9f54e6066f968    452.983060
b38a03232f43b11c9d0788abaf060f7366053b6d    479.269681
70b682d9a3636be23f6120fa9d6b164eb3c6002d      0.557012
715494628a50142ce8cb17191cfe6d0f3cae0934    510.147070
Name: NU_NOTA_MT, dtype: float64

In [43]:
sub.to_csv("answer.csv", header=True)